### Set up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from imutils import paths
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import argparse
import imutils
import time
import dlib
import cv2
import joblib
from sklearn.preprocessing import Normalizer
from keras.models import load_model

In [ ]:
dict = {
    0: 'Dương',
    1: 'Hiếu',
    2: 'Hùng',
    3: 'HĐức',
    4: 'Khôi',
    5: 'Kiên',
    6: 'Linh',
    7: 'Quân',
    8: 'Tâm',
    9: 'Tân',
    10: 'Thắng',
    11: 'Trình',
    12: 'Trường',
    13: 'Tuấn',
    14: 'Vân',
    15: 'Việt Đức',
    16: 'Xuân Anh',
    17: 'Đức'}

### Gradio

In [ ]:
!pip install -q gradio

     |████████████████████████████████| 979 kB 5.4 MB/s 
     |████████████████████████████████| 206 kB 43.6 MB/s 
     |████████████████████████████████| 2.0 MB 33.0 MB/s 
     |████████████████████████████████| 961 kB 18.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 3.6 MB 35.7 MB/s 


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image

detector = dlib.cnn_face_detection_model_v1('./Save_model/face_recognition/mmod_human_face_detector.dat')

def convert_and_trim_bb(image, rect):
  startX = rect.left()
  startY = rect.top()
  endX = rect.right()
  endY = rect.bottom()

  startX = max(0, startX)
  startY = max(0, startY)
  endX = min(endX, image.shape[1])
  endY = min(endY, image.shape[0])

  w = endX - startX
  h = endY - startY
  
  # return our bounding box coordinates
  return (startX, startY, w, h)

def predict(input_img):
  image_path = []
  image_path.append(input_img)

  new_image_list = []
  for image in image_path:
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = detector(rgb, 1)
    boxes = [convert_and_trim_bb(image, r.rect) for r in results]
    crop_image = image

    for (x, y, w, h) in boxes:
      crop_image = image[y:y+h, x:x+w]
  
    new_image_list.append(crop_image)

  image_list = []
  for image in new_image_list:
      resized_image = cv2.resize(image, (160,160), interpolation = cv2.INTER_AREA)
      resized_image = np.expand_dims(resized_image, 0)
      image_list.append(resized_image)
      
  image_list = np.vstack(image_list)

  norm_image_list = []
  for image in image_list:
    image = image.astype("float32")
    mean, std = image.mean(), image.std()
    image = (image - mean) / std

    norm_image_list.append(image)

  norm_image_list = np.asarray(norm_image_list)

  facenet = load_model('./Save_model/face_recognition/facenet_keras.h5')

  features = facenet.predict(norm_image_list)

  normalizer = Normalizer(norm='l2')
  features = normalizer.transform(features)

  random_forest = joblib.load("./Save_model/face_recognition/random_forest.joblib")

  predictions = random_forest.predict_proba(features)

  if predictions[0].max() < 0.65:
    return "This person is unknown."
  else:
    return "This is: {} ({:.2f}%)".format(dict[np.argmax(predictions[0])], predictions[0].max()*100)

iface = gr.Interface(predict, inputs=gr.inputs.Image(shape=(1920, 1080)), outputs="text")

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://19866.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


KeyboardInterrupt: ignored